In [ ]:
#@title Copyright 2025 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title Install Dependencies

!pip install mapclassify xyzservices

from google.colab import auth
from google.cloud import bigquery


# import numpy as np
import pandas as pd
import geopandas as gpd

import folium
import mapclassify
import xyzservices

tiles =  xyzservices.TileProvider.from_qms("Google Satellite Hybrid")

### 1. Authenticate and Initialize EE and BQ



In [ ]:
# Change to a project where you have EE access and EE User Role as well as BigQuery User.
project_id = "your-project-id"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}


auth.authenticate_user()
client = bigquery.Client(project=project_id)

### 2. Create a GeoDataFrame of Republic of the Congo Départements

This data comes from the [Overture Maps data in BigQuery's Public Data Catalog](https://https://carto.com/blog/overture-maps-data-now-on-the-cloud-use-it-with-carto) which is maintained by Google's partner, [Carto](https://carto.com/)

In [ ]:
query = """  SELECT
   *,
   ST_REGIONSTATS(
     geometry,
     'ESA/WorldCover/v200/2021',
     'Map == 10',
     options => JSON '{"scale": 1e3}'
   ).mean as mean
 FROM `bigquery-public-data.overture_maps.division_area` where country = 'CG' and subtype = 'region'
 """

# Use the BQ client to retrieve the Republic of the Congo Départements into a Pandas DataFrame
congo = pd.read_gbq(query,project_id=project_id)

# Convert the DataFrame to a GeoDataFrame, using the geometry column
congo_mean_forest_gdf = gpd.GeoDataFrame(congo, geometry=gpd.GeoSeries.from_wkt(congo.geometry), crs='epsg:4326')


In [ ]:
m = congo_mean_forest_gdf.explore(
    "mean",
    cmap="RdYlGn",
    scheme="NaturalBreaks",
    tooltip=["mean"],
    tiles=tiles,
    attr="Google Maps",
    name="Mean Forested %")

m

In [ ]:
congo_mean_forest_gdf